In [68]:
import csv
import json
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import string
import pickle
import collections
import matplotlib.pyplot as plt
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

lmtz = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\boom-\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import os
os.getcwd()

'C:\\Users\\boom-\\Documents\\GitHub\\stat628-module3-group4-fall2022-uwmadison\\code\\Review Analysis'

### 1. Import review_with_rating from EDA part

In [90]:
bars_review = pd.read_csv("../../data/bars_list_ca.csv").drop(columns=['Unnamed: 0'])
bars_review = pd.DataFrame(data = bars_review, columns = ["business_id","stars_x","text"])

In [53]:
bars_review

,stars_x,text
0,5.0,We were a bit weary about trying the Shellfish...
1,5.0,"Great, beautiful spot to do wine tasting. I lo..."
2,3.0,This was a tough one! On the merits of the w...
3,5.0,"I love trying fresh seafood on piers, wharfs a..."
4,5.0,"Super delish!! No frills! Just great sea food,..."
...,...,...
44688,5.0,Now my favorite brunch spot in Santa Barbara. ...
44689,5.0,The combo meat plate was so good and tons of f...
44690,2.0,My table got so many weird looks from the empl...
44691,5.0,"If I could give more than 5 stars, I would! I ..."


In [6]:
review_with_rating = pd.read_csv("../../data/review_with_rating.csv")

In [9]:
review_with_rating

,rating,review_words
0,5.0,"['bit', 'weary', 'trying', 'shellfish', 'compa..."
1,5.0,"['great', 'beautiful', 'spot', 'wine', 'tastin..."
2,3.0,"['tough', 'one', 'merit', 'wine', 'fairly', 'a..."
3,5.0,"['love', 'trying', 'fresh', 'seafood', 'pier',..."
4,5.0,"['super', 'delish', 'frill', 'great', 'sea', '..."
...,...,...
44688,5.0,"['favorite', 'brunch', 'spot', 'santa', 'barba..."
44689,5.0,"['combo', 'meat', 'plate', 'good', 'ton', 'foo..."
44690,2.0,"['table', 'got', 'many', 'weird', 'look', 'emp..."
44691,5.0,"['could', 'give', '5', 'star', 'would', 'reser..."


### 2. Get the adjectives from review_words

In [11]:
nltk.pos_tag(["tough","got","star"])

[('tough', 'JJ'), ('got', 'VBD'), ('star', 'NN')]

In [22]:
adjectives = []
for row in range(len(review_with_rating)):
    words = ast.literal_eval(review_with_rating["review_words"][row])
    # put category for each word
    tag = nltk.pos_tag(words)
    # select only adjectives
    adj = [word for word, tag in tag if tag == "JJ"]
    adjectives.append(adj)

In [23]:
adjectives

[['weary',
  'shellfish',
  'many',
  'monterey',
  'feast',
  'captive',
  'high',
  'perennial',
  'good',
  'initial',
  'small',
  'plan',
  'peak',
  'special',
  'local',
  'awesome',
  'great',
  'hearty',
  'good',
  'much',
  'shrimp',
  'bisque',
  'prattling',
  'glorious',
  'white',
  'thick',
  'separate',
  'good',
  'great',
  'satisfied',
  'special',
  'overall',
  'great',
  'favorite',
  'safe'],
 ['great', 'beautiful', 'wine', 'tried', 'great', 'good'],
 ['tough',
  'average',
  'understand',
  'lable',
  'available',
  'wonderful',
  'nicer',
  'enjoy',
  'super',
  'expensive',
  'free',
  'wine',
  'winery',
  'give',
  'complimentary',
  'carr',
  'new',
  'wine',
  'limited',
  'beautiful',
  'avoid',
  'much',
  'wine',
  'urban'],
 ['fresh',
  'wharf',
  'fresh',
  'gorgeous',
  'top',
  'beautiful',
  'wrong',
  'good',
  'lunchearly',
  'table',
  'outdoor',
  'right',
  'sunscreen',
  'local',
  'salad',
  'steal',
  'huge',
  'delicious',
  'fresh',
  's

### 3. Create dictionary counting adjectives from reviews

In [37]:
# Collect all adjective
all_adj = [adj for i in adjectives for adj in i]
adj_freq = dict(collections.Counter(all_adj))

In [39]:
adj_freq

{'weary': 18,
 'shellfish': 403,
 'many': 2894,
 'monterey': 6,
 'feast': 10,
 'captive': 3,
 'high': 1662,
 'perennial': 3,
 'good': 23313,
 'initial': 86,
 'small': 3718,
 'plan': 26,
 'peak': 45,
 'special': 3438,
 'local': 3279,
 'awesome': 2563,
 'great': 25744,
 'hearty': 31,
 'much': 3311,
 'shrimp': 774,
 'bisque': 26,
 'prattling': 1,
 'glorious': 30,
 'white': 1475,
 'thick': 265,
 'separate': 176,
 'satisfied': 178,
 'overall': 2519,
 'favorite': 3598,
 'safe': 242,
 'beautiful': 2286,
 'wine': 3433,
 'tried': 604,
 'tough': 325,
 'average': 571,
 'understand': 152,
 'lable': 1,
 'available': 826,
 'wonderful': 2756,
 'nicer': 52,
 'enjoy': 396,
 'super': 1966,
 'expensive': 933,
 'free': 1386,
 'winery': 95,
 'give': 1126,
 'complimentary': 442,
 'carr': 23,
 'new': 3196,
 'limited': 234,
 'avoid': 106,
 'urban': 111,
 'fresh': 6068,
 'wharf': 122,
 'gorgeous': 415,
 'top': 2239,
 'wrong': 1125,
 'lunchearly': 5,
 'table': 5896,
 'outdoor': 1084,
 'right': 2246,
 'sunscreen

In [40]:
len(adj_freq)

18020

### 4. Create matrix of frequency

In [65]:
corpus = pd.DataFrame(data = list(bars_review["text"]), columns = ["text"])
vectorizer = CountVectorizer(vocabulary = list(adj_freq.keys()), min_df = 0)
freq_matrix = pd.DataFrame(data = vectorizer.fit_transform(corpus["text"].values).toarray(), columns = vectorizer.get_feature_names())

In [64]:
corpus

,0
0,We were a bit weary about trying the Shellfish...
1,"Great, beautiful spot to do wine tasting. I lo..."
2,This was a tough one! On the merits of the w...
3,"I love trying fresh seafood on piers, wharfs a..."
4,"Super delish!! No frills! Just great sea food,..."
...,...
44688,Now my favorite brunch spot in Santa Barbara. ...
44689,The combo meat plate was so good and tons of f...
44690,My table got so many weird looks from the empl...
44691,"If I could give more than 5 stars, I would! I ..."


In [66]:
freq_matrix

,weary,shellfish,many,monterey,feast,captive,high,perennial,good,initial,...,not_fear,focusing,cassava,privileged,tupperware,biodegradable,trendyromantic,delicious30,agh,hasslefree
0,1,1,1,1,1,1,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
44689,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
44690,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Fit the model using Naive Bayes classifier

In [69]:
# First we classify stars to positive/negative (4 stars or more are positive)
bars_review["stars_x"] = np.where(bars_review["stars_x"] >= 4, "positive", "negative")

In [70]:
bars_review

,stars_x,text
0,positive,We were a bit weary about trying the Shellfish...
1,positive,"Great, beautiful spot to do wine tasting. I lo..."
2,negative,This was a tough one! On the merits of the w...
3,positive,"I love trying fresh seafood on piers, wharfs a..."
4,positive,"Super delish!! No frills! Just great sea food,..."
...,...,...
44688,positive,Now my favorite brunch spot in Santa Barbara. ...
44689,positive,The combo meat plate was so good and tons of f...
44690,negative,My table got so many weird looks from the empl...
44691,positive,"If I could give more than 5 stars, I would! I ..."


In [72]:
model = MultinomialNB()
model.fit(freq_matrix, bars_review['stars_x'])

MultinomialNB()

In [79]:
# Test the prediction performance

def prediction(adj):
    return model.predict(vectorizer.transform([adj]))[0]

In [81]:
prediction("worst")

'negative'

In [82]:
adj_classifier = {}
for adj in adj_freq.keys():
    adj_classifier[adj] = prediction(adj)

In [83]:
adj_classifier

{'weary': 'positive',
 'shellfish': 'positive',
 'many': 'positive',
 'monterey': 'positive',
 'feast': 'positive',
 'captive': 'negative',
 'high': 'positive',
 'perennial': 'positive',
 'good': 'positive',
 'initial': 'positive',
 'small': 'positive',
 'plan': 'positive',
 'peak': 'positive',
 'special': 'positive',
 'local': 'positive',
 'awesome': 'positive',
 'great': 'positive',
 'hearty': 'positive',
 'much': 'positive',
 'shrimp': 'positive',
 'bisque': 'positive',
 'prattling': 'positive',
 'glorious': 'positive',
 'white': 'positive',
 'thick': 'positive',
 'separate': 'positive',
 'satisfied': 'positive',
 'overall': 'positive',
 'favorite': 'positive',
 'safe': 'positive',
 'beautiful': 'positive',
 'wine': 'positive',
 'tried': 'positive',
 'tough': 'positive',
 'average': 'negative',
 'understand': 'negative',
 'lable': 'negative',
 'available': 'positive',
 'wonderful': 'positive',
 'nicer': 'positive',
 'enjoy': 'positive',
 'super': 'positive',
 'expensive': 'positive'

### 6. Get the nouns from review_words and count

In [84]:
nouns = []
for row in range(len(review_with_rating)):
    words = ast.literal_eval(review_with_rating["review_words"][row])
    # put category for each word
    tag = nltk.pos_tag(words)
    # select only adjectives
    noun = [word for word, tag in tag if tag == "NN"]
    nouns.append(noun)

In [85]:
# Collect all nouns
all_noun = [noun for i in nouns for noun in i]
noun_freq = dict(collections.Counter(all_noun))

In [86]:
noun_freq

{'bit': 3072,
 'company': 510,
 'see': 732,
 'cannery': 2,
 'audience': 15,
 'substandard': 8,
 'fare': 169,
 'price': 4367,
 'review': 2833,
 'yelp': 1136,
 'observation': 12,
 'place': 23142,
 'visit': 2399,
 'hour': 4584,
 'minute': 3155,
 'dinner': 6079,
 'rush': 176,
 'rock': 971,
 'crab': 3401,
 'lb': 66,
 'california': 562,
 'salad': 3758,
 'choice': 1670,
 'soupchowder': 1,
 'look': 1206,
 'tray': 85,
 'wife': 2135,
 'saladchowder': 1,
 'combo': 339,
 'portion': 2394,
 'start': 814,
 'chowder': 3653,
 'side': 2957,
 'dish': 2597,
 'juicy': 507,
 'buttery': 252,
 'chunk': 285,
 'meat': 1928,
 'crack': 91,
 'shell': 456,
 'splinter': 2,
 'sign': 391,
 'perfection': 771,
 'amount': 1077,
 'work': 1373,
 'youre': 2353,
 'meal': 4381,
 'value': 367,
 'sbsc': 13,
 'trip': 1331,
 'proposition': 6,
 'spot': 4156,
 'wine': 9655,
 'merit': 7,
 'winery': 809,
 'label': 37,
 'premium': 42,
 'week': 1266,
 'location': 2984,
 'outdoor': 846,
 'patio': 2367,
 'staff': 7044,
 'super': 1006,
 '

In [88]:
collections.Counter(all_noun).most_common(100)

[('food', 29525),
 ('place', 23142),
 ('service', 18685),
 ('time', 14450),
 ('restaurant', 10491),
 ('wine', 9655),
 ('barbara', 7913),
 ('bar', 7521),
 ('staff', 7044),
 ('order', 6695),
 ('santa', 6694),
 ('menu', 6650),
 ('experience', 6497),
 ('dinner', 6079),
 ('night', 6065),
 ('beer', 5509),
 ('drink', 5334),
 ('everything', 4932),
 ('day', 4918),
 ('hour', 4584),
 ('meal', 4381),
 ('price', 4367),
 ('way', 4352),
 ('friend', 4337),
 ('star', 4298),
 ('thing', 4180),
 ('spot', 4156),
 ('area', 4069),
 ('burger', 4016),
 ('flavor', 4005),
 ('seafood', 3842),
 ('server', 3808),
 ('salad', 3758),
 ('clam', 3659),
 ('chowder', 3653),
 ('cocktail', 3445),
 ('lot', 3410),
 ('crab', 3401),
 ('table', 3288),
 ('try', 3205),
 ('minute', 3155),
 ('bit', 3072),
 ('taste', 3061),
 ('view', 3014),
 ('sauce', 3008),
 ('wait', 2997),
 ('get', 2997),
 ('location', 2984),
 ('side', 2957),
 ('plate', 2895),
 ('reservation', 2888),
 ('chicken', 2886),
 ('selection', 2835),
 ('review', 2833),
 ('l

#### We still need to pick the words that are useful for the analysis

#### Merge the data having 1 business id for each row (combine reviews)

In [111]:
business = pd.read_csv("../../data/bars_list.csv").drop(columns = ["Unnamed: 0"])

In [122]:
business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours
0,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,AB,T5J 1B9,53.546045,-113.499169,4.0,40,0,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
1,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"{'Smoking': ""u'no'"", 'NoiseLevel': ""u'average'...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ..."
2,lk9IwjZXqUMqqOhM774DtQ,Caviar & Bananas,2031 Broadway,Nashville,TN,37203,36.148371,-86.798895,3.5,159,0,"{'RestaurantsTakeOut': 'True', 'RestaurantsRes...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
3,L_TT0BFmFwORAMaA83K54A,Village Tap Room,838 Broad Ripple Ave,Indianapolis,IN,46220,39.869911,-86.143577,2.5,23,0,"{'Alcohol': ""u'none'"", 'BestNights': ""{'monday...",NaN
4,qfWJmJ0g96eM_fWma3ja0g,Doc Magrogan's Oyster House - West Chester,117 E Gay St,West Chester,PA,19380,39.961542,-75.603604,3.0,114,0,"{'WiFi': ""u'no'"", 'RestaurantsAttire': ""u'casu...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3432,2jFOIBICinZkepcO60rN0Q,Tulane Ave Bar & Grill,3813 Tulane Ave,New Orleans,LA,70119,29.967547,-90.102695,4.0,6,0,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...",NaN
3433,XhZEh336ceIy_nphBm3pFA,Sports Legends Bar & Grill,2571 Drew St,Clearwater,FL,33765,27.967356,-82.731122,3.5,111,0,"{'OutdoorSeating': 'True', 'RestaurantsAttire'...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
3434,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,CA,93105,34.429154,-119.717222,5.0,19,1,"{'BusinessParking': ""{'garage': False, 'street...","{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ..."
3435,IeSD0nMKRFYUTnR5nZH1CQ,HighWire Lounge,14 S Arizona Ave,Tucson,AZ,85701,32.221828,-110.967969,3.5,111,1,"{'BusinessParking': ""{'garage': False, 'street...","{'Tuesday': '17:0-2:0', 'Wednesday': '17:0-2:0..."


In [98]:
review_i = []
bus_id = []

for row in range(len(bars_review)):
    if bars_review["business_id"][row] not in bus_id:
        review_i.append(row)
        bus_id.append(bars_review["business_id"][row])

In [102]:
len(review_i)


163

In [104]:
combined_review = []
for i in range(len(review_i)):
    if i < len(review_i)-1:
        combined_review.append(" connection ".join(list(bars_review["text"][review_i[i]:review_i[i+1]])))
    else: # for the last review row
        combined_review.append(" connection ".join(list(bars_review["text"][review_i[i]:])))

In [116]:
combined_review_df

,0
0,We were a bit weary about trying the Shellfish...
1,"Great, beautiful spot to do wine tasting. I lo..."
2,Reviewing the food.\nHard to justify $16 for a...
3,"First time eating here today, was fantastic! T..."
4,In Montecito everything is so Montecito. This ...
...,...
158,For a small wine shop they stock some very ext...
159,Looks like an attempt to class up the grimy Cr...
160,We loved this place! We were greeted by a beau...
161,Ate there for the first time today and really ...


In [114]:
combined_review_df = pd.DataFrame(combined_review)
result_df = pd.concat([business["business_id"],business["stars"],combined_review_df], axis = 1)
result_df.columns = ["business_id","stars","combined_review"]

In [115]:
result_df

,business_id,stars,combined_review
0,WKMJwqnfZKsAae75RMP6jA,4.0,We were a bit weary about trying the Shellfish...
1,QdN72BWoyFypdGJhhI5r7g,4.0,"Great, beautiful spot to do wine tasting. I lo..."
2,lk9IwjZXqUMqqOhM774DtQ,3.5,Reviewing the food.\nHard to justify $16 for a...
3,L_TT0BFmFwORAMaA83K54A,2.5,"First time eating here today, was fantastic! T..."
4,qfWJmJ0g96eM_fWma3ja0g,3.0,In Montecito everything is so Montecito. This ...
...,...,...,...
3432,2jFOIBICinZkepcO60rN0Q,4.0,NaN
3433,XhZEh336ceIy_nphBm3pFA,3.5,NaN
3434,Bo-GGwQNcLVwyKeYkbfAYg,5.0,NaN
3435,IeSD0nMKRFYUTnR5nZH1CQ,3.5,NaN


In [120]:
bars_review["business_id"].unique()

array(['SZU9c8V2GuREDN5KgyHFJw', 'eL4lyE7LNoXEMvpcJ8WNVw',
       '-ujBP1Dw0j1-Ffaz97-LXQ', '1FURjeGJi_LBXcJQg8eskw',
       'EaTbY8HFwwF7i3SjVbHGYw', 'ywXa1pDqHPe9tOPx0yJFUg',
       'H-1qpp_77KggOAr9htUrEw', 'MAIfD4kJDCydgoD1l2b_vw',
       '2nk4hq2oRfym8ROQ5iS47g', 'vsrryxBR2Jykl71qa1H6SQ',
       'PPcz-0UtIJ8O7xDT551y2g', 'DD3TxygdxBxKh9gbjCuLDA',
       'zbrIMldF_O1ZQ0vpUaaa8A', 'fnGZspqyamRremXor0sAyg',
       'ycbmvmBYk4w_ZmdrPL0tBQ', '4xhGQGdGqU60BIznBjqnuA',
       'q75UQjNm6AEK4XNnkDLxTQ', '1xsBRB203LiYLcC3mktfdA',
       'pqF3WkYktbR28uk2PSR7Rg', 'jfRq1KtjEGyPoMo9HZVw2Q',
       '52ov7d0148JvYG78_c-Bsw', 'gdEZfd6RFXYmmXX71RDvbA',
       '4KAuy-zNBxD9QvnkldwEww', '6HTGlttrzCMsuGBHO1ZGiw',
       '9xW7LsJpyhVZFRb6z9xorg', '_dNk5IpkVVyN2nWMXTA2eA',
       'R93mEcQ1pjq7q3LzShi3ow', 'sZHLbLnw1vwHF1xDzuV1iw',
       'GBD-YjcOD1nG0wv4L0Lvfg', 'A7Uwcr1BjasB6BWke1hrfA',
       'U3grYFIeu6RgAAQgdriHww', 'c_a38vt6v2tRq-VjdTZHZw',
       'cAbdvzqtFLaAAMFIyPf2AA', 'LtxG9DTXCFpteVkD4SGBlw

In [118]:
bars_review["business_id"]

0        SZU9c8V2GuREDN5KgyHFJw
1        eL4lyE7LNoXEMvpcJ8WNVw
2        eL4lyE7LNoXEMvpcJ8WNVw
3        SZU9c8V2GuREDN5KgyHFJw
4        SZU9c8V2GuREDN5KgyHFJw
                  ...          
44688    q-1ICFSqs2MwDGfe5tX-SQ
44689    wMZAnrfwV-qHiV5zdTSrzw
44690    KSYONgGtrK0nKXfroB-bwg
44691    fiQxkb1pfPpvrr0oYYYBcA
44692    r2IhvKZQ_wLR5mLBnPOilg
Name: business_id, Length: 44693, dtype: object

In [123]:
business[business["state"] == "CA"]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours
9,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,"{'OutdoorSeating': 'True', 'RestaurantsAttire'...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
18,4xhGQGdGqU60BIznBjqnuA,California Tacos and Taproom,956 Embarcadero Del Norte,Isla Vista,CA,93117,34.411555,-119.855077,4.0,49,0,"{'Music': ""{'dj': False, 'background_music': F...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-22:0', ..."
36,-ujBP1Dw0j1-Ffaz97-LXQ,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,"{'RestaurantsReservations': 'False', 'WiFi': ""...","{'Monday': '0:0-0:0', 'Tuesday': '11:30-17:0',..."
102,MAIfD4kJDCydgoD1l2b_vw,Corks n' Crowns,32 Anacapa St,Santa Barbara,CA,93101,34.414211,-119.688757,4.0,276,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'..."
119,1FURjeGJi_LBXcJQg8eskw,Padaro Beach Grill,3765 Santa Claus Ln,Carpinteria,CA,93013,34.406833,-119.548583,4.5,885,1,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-19:30',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3373,QLHgjBHBoeIUoEvIPvjHnw,Mesa Burger - Santa Barbara,"315 Meigs Rd, Ste F",Santa Barbara,CA,93109,34.400798,-119.722671,4.0,442,1,"{'DriveThru': 'False', 'RestaurantsAttire': ""u...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ..."
3402,r2IhvKZQ_wLR5mLBnPOilg,The Honor Bar,"1255 Coast Village Rd, Ste 101",Santa Barbara,CA,93108,34.421044,-119.641903,4.5,557,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
3405,sz5SMsTOTAshdfsaK98-TA,Jane At The Marketplace,6940 Marketplace Dr,Goleta,CA,93117,34.429750,-119.870280,4.0,291,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-20:30',..."
3408,gkqfm6mTim9Rwopwjayc8w,Five & 1/4,705 Anacapa St,Santa Barbara,CA,93101,34.419813,-119.697266,5.0,27,0,"{'RestaurantsReservations': 'False', 'BikePark...","{'Thursday': '17:15-22:0', 'Friday': '17:15-22..."
